In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import pandas as pd

In [9]:
DIR = BASE_DIR/'scripts_res'/'Algorithms'
DIR

WindowsPath('c:/Users/Nicol/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res/Algorithms')

In [ ]:
** # Don't run all

### ASBO

In [5]:
re_run_ASBO = True
re_run_save_ASBO = True

sol = 'ASBO'
case_name = 'Case1'
runs_number = 100

In [17]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

obj_list = []
x_1_list = []
x_2_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
all_res_list = []

def func(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return -obj,cons

nc_lb = np.array([-1e6,-1e6,-1e6])
nc_ub = np.array([0,0,0])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-10, 5),'x2': (-15, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

if re_run_ASBO == True or re_run_save_ASBO == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=50, n_iter=50,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        con_3_list.extend([optimizer.max['constraint'][2]])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list
        })
        all_res = [item['target'] for item in optimizer.res]
        all_fea = [item['allowed'] for item in optimizer.res]
        modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
        all_res_list.append(modified_res)
        df2 = pd.DataFrame(all_res_list, columns=[list(range(50+50))])

        if re_run_save_ASBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)

{'target': -2.3269307027986507e-06, 'params': {'x1': -4.020306908294415, 'x2': -6.021089227928202}, 'constraint': array([-12.98047541,  -6.97870459, -75.37209556])}
{'target': -1.2414316320263125e-06, 'params': {'x1': -4.016204077101914, 'x2': -6.0081853533781056}, 'constraint': array([-12.9757772 ,  -6.99168336, -75.04848804])}
{'target': -9.294857577576002e-06, 'params': {'x1': -4.030149412284439, 'x2': -5.931229195326578}, 'constraint': array([-12.87093037,  -7.06831631, -73.25561945])}
{'target': -8.420453668672719e-07, 'params': {'x1': -3.991272449512709, 'x2': -6.021287561629923}, 'constraint': array([-13.03874266,  -6.97867435, -75.28389487])}
{'target': -5.61194072186243e-07, 'params': {'x1': -4.001868718965998, 'x2': -5.977839083442418}, 'constraint': array([-12.97410165,  -7.02215917, -74.27510081])}
{'target': -1.8200741352203502e-06, 'params': {'x1': -3.9797643940504295, 'x2': -6.000384355451591}, 'constraint': array([-13.04085557,  -6.9994109 , -74.74455278])}
{'target': -

In [10]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1,con_2,con_3
count,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000
mean,-3.308306e-06,-4.001956,-6.003535,-12.999622,-6.996261,-74.893862
std,3.279421e-06,0.020219,0.036665,0.052176,0.036670,0.888640
min,-1.264534e-05,-4.048116,-6.087129,-13.134667,-7.074598,-76.954594
25%,-4.937524e-06,-4.014358,-6.028332,-13.033779,-7.021768,-75.498356
50%,-2.103970e-06,-4.001847,-6.006006,-12.999197,-6.993965,-74.945828
75%,-6.287762e-07,-3.989981,-5.977752,-12.969412,-6.971643,-74.261048
max,-8.355741e-09,-3.947173,-5.925399,-12.870930,-6.912757,-73.028216


### CBO

In [11]:
re_run_CBO = True
re_run_save_CBO = True
sol = 'CBO'
case_name = 'Case1'
runs_number = 100

In [20]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

obj_list = []
x_1_list = []
x_2_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
all_res_list = []

def func_obj(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    # print(('cons, obj value=', cons, obj))
    return -obj

def func_cons(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return cons

nc_lb = np.array([-1e6,-1e6,-1e6])
nc_ub = np.array([0,0,0])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-10, 5),'x2': (-15, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

if re_run_CBO == True or re_run_save_CBO == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=50, n_iter=50,acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        con_3_list.extend([optimizer.max['constraint'][2]])
        
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list
        })
        all_res = [item['target'] for item in optimizer.res]
        all_fea = [item['allowed'] for item in optimizer.res]
        modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
        all_res_list.append(modified_res)
        df2 = pd.DataFrame(all_res_list, columns=[list(range(50+50))])
        
        if re_run_save_CBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)


{'target': -5.455644286023793e-07, 'params': {'x1': -4.010835022295992, 'x2': -6.004627867016621}, 'constraint': array([-12.98295782,  -6.99531343, -74.94580719])}
{'target': -1.367714610585897e-06, 'params': {'x1': -3.982615065874239, 'x2': -6.00469035300368}, 'constraint': array([-13.03946022,  -6.99515853, -74.85704113])}
{'target': -4.6471311903758557e-07, 'params': {'x1': -3.9907903318434697, 'x2': -5.991113501425182}, 'constraint': array([-13.00953284,  -7.00884409, -74.557428  ])}
{'target': -5.851094302716218e-08, 'params': {'x1': -4.003549014614085, 'x2': -5.9984907519088955}, 'constraint': array([-12.99139272,  -7.00150295, -74.77514197])}
{'target': -3.767704187111443e-06, 'params': {'x1': -3.971920826102246, 'x2': -6.015400446138962}, 'constraint': array([-13.07155879,  -6.98420533, -75.08038939])}
{'target': -1.7163878233421858e-06, 'params': {'x1': -4.007710680185506, 'x2': -5.96384850369189}, 'constraint': array([-12.94842714,  -7.03612177, -73.95966402])}
{'target': -6.

In [12]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
an_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1,con_2,con_3
count,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000
mean,-1.490675e-06,-3.999468,-6.001247,-13.002311,-6.998632,-74.829024
std,1.800810e-06,0.015625,0.019388,0.034277,0.019374,0.475419
min,-6.454068e-06,-4.034069,-6.061543,-13.090269,-7.063755,-76.351321
25%,-2.169182e-06,-4.007271,-6.011225,-13.024181,-7.009614,-75.058007
50%,-5.688084e-07,-4.000364,-6.000469,-12.997657,-6.999529,-74.816405
75%,-2.279502e-07,-3.991452,-5.989945,-12.984829,-6.988616,-74.555718
max,-6.048515e-09,-3.962709,-5.936177,-12.918978,-6.938240,-73.239213


### SCBO

In [17]:
re_run_SCBO = True
re_run_save_SCBO = True
sol = 'SCBO'
case_name = 'Case1'
runs_number = 100

In [6]:
x_1_list = []
x_2_list = []
iterations = []
max_values = []
all_res_list = []

import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

def func(x):
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return -obj,c1,c2,c3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
dim = 2
batch_size = 1
n_init = 50
bo_iter = 50 
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1e6, -1e6, -1e6, ], dtype=torch.float64)
ub =  torch.tensor([0, 0,0 ], dtype=torch.float64)
bounds = torch.tensor([[-10, -15,], [5, 15]])

def objective(x):
    res = func(x)
    target = res[0]
    c1 = res[1]
    c2 = res[2]
    c3 = res[3]
    return target,c1,c2,c3

def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []
            C2_tensor_list = []
            C3_tensor_list = []

            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1, C2,C3 = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], 
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)
                C2_tensor_list.append(C2)
                C3_tensor_list.append(C3)

            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)
            C2 = torch.stack(C2_tensor_list, dim=0)
            C3 = torch.stack(C3_tensor_list, dim=0)

            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)
                c2_model = tr_bo.get_fitted_model(train_X, C2,dim)
                c3_model = tr_bo.get_fitted_model(train_X, C3,dim)
                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model, c2_model,c3_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                C2_next_tensor_list = []
                C3_next_tensor_list = []

                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next, C2_next, C3_next, = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3],  
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)
                    C2_next_tensor_list.append(C2_next)
                    C3_next_tensor_list.append(C3_next)

                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)
                C2_next = torch.stack(C2_next_tensor_list, dim=0)
                C3_next = torch.stack(C3_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next, C2_next,C3_next], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                C2 = torch.cat((C2, C2_next), dim=0)
                C3 = torch.cat((C3, C3_next), dim=0)
                C = torch.cat([C1, C2, C3], dim=-1)
                bool_tensor = (C >= lb.repeat(len(C), 1)) & (C <= ub.repeat(len(C), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                Valid_X = train_X[bool_tensor]
                

                if len(Valid_Y)!=0:
                    max_index = torch.argmax(Valid_Y)  
                else:
                    print(f"{len(train_X)}) No feasible point yet! TR length: {state.length:.2e}")  

                if len(train_X) == n_init + bo_iter:
                    modified_Y = torch.where(bool_tensor, train_Y.flatten(), torch.tensor(float('-inf')))
                    all_res_list.append(modified_Y.tolist())
                    df2 = pd.DataFrame(all_res_list, columns=[list(range(n_init+bo_iter))])

                    if len(Valid_Y)!=0:
                        max_value = Valid_Y[max_index].item()
                        c_x = Valid_X[max_index]
                        print("obj= ",max_value, ' x = ', c_x)
                        max_values.append(max_value)
                        x_1_list.append(c_x[0].item())
                        x_2_list.append(c_x[1].item())
                    else:
                        print("No feasible solutions")
                        max_values.append(None)
                        x_1_list.append(None)
                        x_2_list.append(None)
                    iterations.append(len(train_X))
                    df = pd.DataFrame({
                    'obj': max_values,
                    'x_1': x_1_list,
                    'x_2': x_2_list,
                    'iterations': iterations
                    })
                    if re_run_save_SCBO == True:
                        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                        df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)
                    break

        except Exception as e:
            fixed_length_Y = torch.full((n_init+bo_iter,), float('-inf'))           
            modified_Y = torch.where(bool_tensor, train_Y.flatten(), torch.tensor(float('-inf')))
            fixed_length_Y[:modified_Y.size(0)] = modified_Y
            all_res_list.append(fixed_length_Y.tolist())
            df2 = pd.DataFrame(all_res_list, columns=[list(range(n_init+bo_iter))])

            if len(train_X) == n_init + bo_iter:
                print(e)
            else:
                print(e, "Early Stop")
                if len(Valid_Y)!=0:
                    max_value = Valid_Y[max_index].item()
                    c_x = Valid_X[max_index]
                    print("obj= ",max_value, ' x = ', c_x)
                    max_values.append(max_value)
                    x_1_list.append(c_x[0].item())
                    x_2_list.append(c_x[1].item())
                else:   
                    print("obj= ", None, ' x = ', None)         
                    max_values.append(None)
                    x_1_list.append(None)
                    x_2_list.append(None)
            iterations.append(len(train_X))
            df = pd.DataFrame({
            'obj': max_values,
            'x_1': x_1_list,
            'x_2': x_2_list,
            'iterations': iterations
            })
            if re_run_save_SCBO == True:
                df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)

run_multiple_calculations(num_runs=runs_number)

All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  -2.160384042990608e-06  x =  tensor([0.4011, 0.3010], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  -4.2393646515348153e-07  x =  tensor([0.4003, 0.3006], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  -9.405464372571767e-06  x =  tensor([0.3979, 0.2978], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  -3.747651628356181e-05  x =  tensor([0.3947, 0.3030], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  -7.104435674384405e-06  x =  tensor([0.4027, 0.2999], dtype=torch.float64)
All 

In [18]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,1.000000e+02,100.000000,100.000000,100.000000
mean,-5.373272e-06,0.399957,0.300035,73.740000
std,6.113992e-06,0.001728,0.001562,12.970534
min,-3.747652e-05,0.394684,0.295013,52.000000
25%,-7.047635e-06,0.399072,0.299148,63.000000
50%,-3.762874e-06,0.399906,0.300104,73.000000
75%,-9.912042e-07,0.401074,0.301069,82.250000
max,-4.830361e-09,0.404030,0.303263,100.000000


In [ ]:
**

### trust-constr

In [11]:
re_run_tc = True
re_run_save_tc = True
sol = 'trust-constr'
case_name = 'Case1'
runs_number = 100

In [20]:
def func(x):
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return obj,c1,c2,c3

initial_x1 = []
initial_x2 = []
x_1_list = []
x_2_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []

nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1e6,-1e6,-1e6])
    nc_ub = np.array([0,0,0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x[0]
    lb = [-10,-15] 
    ub = [5,15] 
    bd = Bounds(lb, ub, True) 
    try:
        result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
        print('obj = ', result.fun, ', x = ', result.x)
        obj_list.append(result.fun)
        x_1_list.append(result.x[0])
        x_2_list.append(result.x[1])
        con_1_list.append(result.constr[0][0])
        con_2_list.append(result.constr[0][1])
        con_3_list.append(result.constr[0][2])    
        initial_x1.append(x[0][0])  
        initial_x2.append(x[0][1])  
        nfev_r.append(result.nfev)
        success_r.append(str(result.success))
        message_r.append(result.message)
 
    except Exception as e:
        print(e)
        obj_list.append(None)
        x_1_list.append(None)
        x_2_list.append(None)
        con_1_list.append(None)
        con_2_list.append(None)
        con_3_list.append(None)    
        initial_x1.append(x[0][0])  
        initial_x2.append(x[0][1])  
        nfev_r.append(None)
        success_r.append(None)
        message_r.append(e)        
    df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })   
    if re_run_save_tc == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    # return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run_tc == True or re_run_save_tc == True:
    x_random = initial_random(lb=[-10,-15],ub=[5,15])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


obj =  1.6252162434839503e-11 , x =  [-3.99997541 -6.00011049]
obj =  6.416425148284175e-13 , x =  [-3.99999517 -6.00002201]
obj =  4.0203436097784545e-10 , x =  [-3.99987984 -6.00055143]
obj =  1.623599829184712e-11 , x =  [-3.99997549 -6.0001105 ]
obj =  6.424844200901382e-13 , x =  [-3.99999516 -6.00002201]
obj =  1.620139491782229e-11 , x =  [-3.9999755  -6.00011036]
obj =  1.6243867014632098e-11 , x =  [-3.99997545 -6.00011049]
obj =  1.6270733790415293e-11 , x =  [-3.99997528 -6.00011045]
obj =  1.0036141902714577e-08 , x =  [-3.99939981 -6.00275529]
obj =  6.535012256039548e-13 , x =  [-3.99999515 -6.00002222]
obj =  2.484942518802391e-07 , x =  [-3.99700633 -6.01370389]
obj =  4.176778165706565e-15 , x =  [-3.99999915 -6.00000094]
obj =  1.623557806439915e-11 , x =  [-3.99997543 -6.00011044]
obj =  2.6039603708060234e-14 , x =  [-3.99999898 -6.00000439]
obj =  1.6242806488432772e-11 , x =  [-3.99997545 -6.00011048]
obj =  1.004281316094566e-08 , x =  [-3.99939922 -6.00275587]
o

In [12]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df_cleaned = df.dropna(subset=['obj'])
df_cleaned.describe()

NaN count in column 'obj': 3


,obj,x_1,x_2,con_1,con_2,con_3,initial_x1,initial_x2,nfev
count,9.700000e+01,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,4.975974e-06,-3.996873,-6.013559,-13.019813,-6.986342,-75.122824,-2.470513,-2.322883,642.680412
std,2.378634e-05,0.013784,0.059496,0.087063,0.059967,1.419082,4.053359,8.099547,485.029800
min,4.176778e-15,-3.999999,-6.296964,-13.434618,-6.999999,-81.884222,-9.873275,-14.609480,54.000000
25%,1.604721e-11,-3.999976,-6.000111,-13.000161,-6.999890,-74.802580,-5.604652,-9.381404,246.000000
50%,1.623721e-11,-3.999975,-6.000110,-13.000160,-6.999890,-74.802573,-2.458319,-2.497397,402.000000
75%,1.647336e-11,-3.999975,-6.000110,-13.000158,-6.999889,-74.802566,1.126326,3.578129,1173.000000
max,1.190403e-04,-3.931173,-6.000001,-13.000003,-6.700667,-74.800020,4.907780,14.655291,1512.000000


### slsqp

In [9]:
re_run_slsqp = True
re_run_save_slsqp = True
sol = 'slsqp'
case_name = 'Case1'
runs_number = 100

In [23]:
initial_x1 = []
initial_x2 = []
x_1_list = []
x_2_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return obj,c1,c2,c3

def recyc(x):
    nc_lb = np.array([-1e6,-1e6,-1e6])
    nc_ub = np.array([0,0,0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x[0]
    lb = [-10,-15] 
    ub = [5,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0, method='slsqp', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
 
    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save_slsqp == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x
    
def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run_slsqp == True or re_run_save_slsqp == True:
    x_random = initial_random(lb=[-10,-15],ub=[5,15])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


obj =  1.2687995058116856e-16 , x =  [-4.00000007 -5.9999997 ]
obj =  3.406509523185186e-08 , x =  [-3.99855412 -6.00472189]
obj =  2.302209347916708e-08 , x =  [-4.00211912 -6.00166047]
obj =  1.3176627171685627e-08 , x =  [-4.00047636 -6.00330927]
obj =  5.22165056839467e-19 , x =  [-4.00000001 -6.00000001]
obj =  3.295121280820738e-17 , x =  [-4.00000008 -6.00000008]
obj =  9.341630258757935e-08 , x =  [-4.00228154 -6.00795318]
obj =  1.5930903047198795e-16 , x =  [-3.99999982 -5.99999986]
obj =  1.2346256994896392e-10 , x =  [-4.00012122 -5.99977123]
obj =  8.106885689505156e-16 , x =  [-4.00000042 -5.99999985]
obj =  1.467378484598438e-09 , x =  [-4.00050435 -5.99944938]
obj =  8.312709294811813e-19 , x =  [-4.00000001 -5.99999999]
obj =  2.0025262253091905e-07 , x =  [-3.99747263 -6.01243692]
obj =  3.35683719728511e-07 , x =  [-3.99167227 -5.99502896]
obj =  2.1475100990422016e-19 , x =  [-4.00000001 -6.00000001]
obj =  4.975007735076881e-07 , x =  [-4.00990569 -5.99256632]
obj 

In [10]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['success'] == False).sum()
print(f"Count of rows where 'success' is False: {false_count}")
df_filtered = df[df['success'] != False]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'success' is False: 0


,obj,x_1,x_2,initial_x1,initial_x2,nfev
count,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000
mean,4.150893e-06,-3.999626,-5.998156,-2.545412,-1.877065,36.610000
std,1.811933e-05,0.007394,0.059589,4.047803,8.374339,9.093115
min,9.877712e-20,-4.043457,-6.321636,-9.873275,-14.609480,15.000000
25%,2.908929e-17,-4.000259,-6.001163,-6.058449,-9.139741,33.000000
50%,3.431384e-10,-4.000000,-6.000000,-2.501106,-2.100974,33.000000
75%,4.925668e-08,-3.999999,-5.999997,1.088579,4.624304,43.000000
max,1.149885e-04,-3.949735,-5.708640,4.907780,14.655291,61.000000


### PSO

In [7]:
re_run_PSO = True
re_run_save_PSO = True
sol = 'PSO'
case_name = 'Case1'
runs_number = 100

In [14]:
from pyswarm import pso

x_1_list = []
x_2_list = []
obj_list = []
iterations=[]
state_list = []
itera = 0
def func(x):
    global itera
    obj, c1, c2, c3 = Case1(x)
    cons = [-c1,-c2,-c3]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

lb=[-10,-15]
ub=[5,15]
if re_run_PSO == True or re_run_save_PSO == True:
    for num in range(runs_number):
        xopt, fopt = pso(lambda x:func(x)[0], lb, ub,f_ieqcons=lambda x:func(x)[1:2], swarmsize=50, omega=0.5, phip=0.5, phig=0.5, maxiter=50, minstep=1e-8,minfunc=1e-8, debug=False)
        state = np.all(np.array(func(xopt)[1:])>= 0)
        itera=itera-1         
        print( 'fopt =', fopt, 'xopt = ', xopt, 'state = ', state)   
        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(fopt)
        x_1_list.append(xopt[0])
        x_2_list.append(xopt[1])
        state_list.append(state)
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'iterations': iterations,
        'state':state_list
        })
        if re_run_save_PSO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


Stopping search: Swarm best objective change less than 1e-08
fopt = 2.273718285837235e-09 xopt =  [-3.99955396 -6.00111827] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 6.045914024263069e-08 xopt =  [-3.99638562 -6.00146906] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 4.3353334222760574e-07 xopt =  [-4.00006583 -5.98024746] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 1.2212031353409215e-09 xopt =  [-4.00051935 -5.99985792] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 2.0431619313558552e-09 xopt =  [-4.00067541 -5.99988114] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 1.5282820950781243e-08 xopt =  [-3.99821854 -5.99897036] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = 1.050532485212042e-08 xopt =  [-4.00140635 -5.99875761] state =  True
Stopping search: Swarm best ob

In [8]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['obj'] == 1e+100).sum()
print(f"Count of rows where 'obj' is 1e+100: {false_count}")
df_filtered = df[df['obj'] != 1e+100]
df_filtered = df_filtered[df_filtered['state'] == True]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'obj' is 1e+100: 0


,obj,x_1,x_2,iterations
count,1.000000e+02,100.000000,100.000000,100.000000
mean,9.130927e-08,-3.999790,-5.999705,937.160000
std,4.160811e-07,0.003642,0.005448,125.758123
min,9.651526e-12,-4.008959,-6.029438,500.000000
25%,2.244078e-09,-4.000830,-6.001309,881.000000
50%,7.039108e-09,-4.000175,-5.999868,946.000000
75%,2.957768e-08,-3.999331,-5.998725,1025.500000
max,3.535320e-06,-3.972534,-5.980247,1170.000000


### GA

In [5]:
re_run_GA = True
re_run_save_GA = True
sol = 'GA'
case_name = 'Case1'
runs_number = 100

In [6]:
from sko.GA import GA
from sko.operators import crossover

x_1_list = []
x_2_list = []
obj_list = []
iterations=[]
state_list = []
itera = 0
def func(x):
    global itera
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    itera=itera+1
    return obj,c1,c2,c3

lb=[-10,-15]
ub=[5,15]
if re_run_GA == True or re_run_save_GA == True:
    for num in range(runs_number):
        ga = GA(func=lambda x:func(x)[0], n_dim=2, size_pop=50, max_iter=50, lb =[-10,-15],ub=[5,15], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[1],lambda x:func(x)[2],lambda x:func(x)[3]])
        ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
        best_x, best_y = ga.run()
        state = np.all(np.array(func(best_x)[1:])<= 0)
        itera=itera-1 
        print( 'obj =', best_y, 'x = ', best_x,'state = ', state)

        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(best_y[0])
        x_1_list.append(best_x[0])
        x_2_list.append(best_x[1])
        state_list.append(state)
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'iterations': iterations,
        'state':state_list
        })
        if re_run_save_GA == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


obj = [2.48456006e-07] x =  [-4.00641411 -6.00768421] state =  True
obj = [0.00067036] x =  [-4.00061579 -6.77674093] state =  True
obj = [5.54368629e-06] x =  [-3.96623116 -5.97931236] state =  True
obj = [0.0002967] x =  [-4.25834428 -6.00819908] state =  True
obj = [0.00177566] x =  [-3.43661205 -5.42687515] state =  True
obj = [1.20811263e-06] x =  [-4.00103813 -6.03290882] state =  True
obj = [0.00155928] x =  [-3.40768666 -6.00372281] state =  True
obj = [3.5101446e-06] x =  [-4.02761737 -5.98959545] state =  True
obj = [6.8966225e-05] x =  [-4.12425529 -6.01766622] state =  True
obj = [4.86411014e-07] x =  [-3.99764524 -6.02038603] state =  True
obj = [1.46047339e-07] x =  [-3.99777142 -5.98943703] state =  True
obj = [8.26974371e-06] x =  [-4.00652839 -6.08527772] state =  True
obj = [2.27546792e-06] x =  [-4.00882536 -6.04166981] state =  True
obj = [1.11305268e-09] x =  [-3.99962969 -5.99932678] state =  True
obj = [0.00572696] x =  [-4.14414426 -3.74807823] state =  True
obj

In [6]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df_filtered = df[df['state'] == True]
df_filtered.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,1.000000e+02,100.000000,100.000000,100.0
mean,2.127737e-04,-4.014404,-5.997428,10001.0
std,6.542918e-04,0.156255,0.308095,0.0
min,1.078137e-09,-4.606992,-6.929831,10001.0
25%,8.915962e-07,-4.027655,-6.032700,10001.0
50%,7.822410e-06,-4.002166,-6.000166,10001.0
75%,1.143000e-04,-3.971714,-5.978306,10001.0
max,5.726958e-03,-3.407687,-3.748078,10001.0
